In [1]:
import numpy as np
import cv2

In [2]:
net = cv2.dnn.readNet("yolov3.weights","yolov3.cfg")

In [3]:
classes = []
with open('coco.names','r') as f:
    classes = f.read().splitlines()

In [4]:
img = cv2.imread('img.jpg')
height,width,_ = img.shape

In [5]:
blob = cv2.dnn.blobFromImage(img,1/255,(416,416),(0,0,0),swapRB=True , crop=False)

In [6]:
net.setInput(blob)
output_layers_names=net.getUnconnectedOutLayersNames()
layersOutputs = net.forward(output_layers_names)

In [7]:
boxes=[]
confidences = []
class_ids = []

In [8]:
for output in layersOutputs:
    for detection in output:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            x_center = int(detection[0]*width)
            y_center = int(detection[1]*height)
            w = int(detection[2]*width)
            h = int(detection[3]*height)  
            
            x = int(x_center - w/2)
            y = int(y_center - h/2)    
            
            boxes.append([x,y,w,h])
            confidences.append(float(confidence))
            class_ids.append(class_id)
            
indexes = cv2.dnn.NMSBoxes(boxes,confidences,0.5,0.4)
font = cv2.FONT_HERSHEY_PLAIN
colors = np.random.uniform(0,255,size=(len(boxes),3))

for i in indexes.flatten():
    x,y,w,h = boxes[i]
    label = str(classes[class_ids[i]])
    confidence = str(round(confidences[i],2))
    color = colors[i]
    
    cv2.rectangle(img, (x,y), (x+w, y+h), color, 2)
    cv2.putText(img, label + " " + confidence, (x, y+20), font, 2, (255,255,255), 2)

In [9]:
cv2.imshow("image",img)
cv2.waitKey(0)
cv2.destroyAllWindows()